<a href="https://colab.research.google.com/github/ahmedSadoun/colab-project/blob/main/Xceptionskincancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

100% 5.20G/5.20G [01:13<00:00, 24.5MB/s]



In [ ]:
import zipfile
zf = "/content/skin-cancer-mnist-ham10000.zip"
target_dir = "/content/dataset/data"
zfile = zipfile.ZipFile(zf)
zfile.extractall(target_dir)

In [ ]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from tqdm import tqdm
import tensorflow as tf
import keras
import numpy as np
import cv2
import seaborn as sns
import random
from PIL import Image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import Xception,VGG16,ResNet50
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from numpy.random import seed

In [ ]:
# Reading the data
path = '/content/dataset/data/'
images_path = {os.path.splitext(os.path.basename(x))[0]:x for x in glob.glob(os.path.join(path,'*','*.jpg'))}

In [ ]:
# Encoding target values
lesion_type_dict = {
    'nv':'Melanocytic Nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
    
}


In [ ]:
# Reading Metadata csv file
skin_df = pd.read_csv('/content/dataset/data/HAM10000_metadata.csv')

In [ ]:
# Adding extra columns and appending the required created columns in final dataframe
skin_df['path'] = skin_df['image_id'].map(images_path.get)
skin_df['cell_type'] = skin_df['dx'].map(lesion_type_dict.get)
skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes

In [ ]:
# Checking the dataframe info.
skin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   lesion_id      10015 non-null  object 
 1   image_id       10015 non-null  object 
 2   dx             10015 non-null  object 
 3   dx_type        10015 non-null  object 
 4   age            9958 non-null   float64
 5   sex            10015 non-null  object 
 6   localization   10015 non-null  object 
 7   path           10015 non-null  object 
 8   cell_type      10015 non-null  object 
 9   cell_type_idx  10015 non-null  int8   
dtypes: float64(1), int8(1), object(8)
memory usage: 714.1+ KB


In [ ]:
# Image Reading and Resizing.
IMAGE_SIZE = 71
def read_img(img_path):
  img = cv2.imread(img_path,cv2.IMREAD_COLOR)
  img = cv2.resize(img,(IMAGE_SIZE,IMAGE_SIZE))
  return img
train_img = []
for img_name in tqdm(skin_df['path'].values):
  train_img.append(read_img(img_name))

100%|██████████| 10015/10015 [01:26<00:00, 115.38it/s]


In [ ]:
# Converting it into float arrays and normalizing it
X = np.array(train_img,np.float32)/255
print(X.shape)

(10015, 71, 71, 3)


In [ ]:
# Creating Target
y = np.array(skin_df['cell_type_idx'])
print(y.shape)

(10015,)


In [ ]:
# Splitting the data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.10,random_state=28,stratify=y)
x_train,x_val,y_train,y_val =   train_test_split(x_train,y_train,test_size=0.10,random_state=28,stratify=y_train)

In [ ]:
# One-Hot Encoding values
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes = 7)
y_val = to_categorical(y_val , num_classes=7)



In [ ]:
# Defining base model using Xception module from Keras
training_shape = (71,71,3)
base_model = Xception(include_top=False,weights='imagenet',input_shape = training_shape)
for layer in base_model.layers:
    layer.trainable = True                         # Training all layers from scratch

83689472/83683744 [==============================] - 0s 0us/step


In [ ]:
#Adding layers at end
n_classes = 7
model = base_model.output
model = Flatten()(model)
model = Dense(128)(model)
model = Dropout(0.5)(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
output = Dense(n_classes, activation='softmax')(model)
model = Model(inputs=base_model.input, outputs=output)

In [ ]:
# Compiling the model
optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])

In [ ]:
# Defining callback Methods
n_epoch = 300

early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1, 
                           mode='auto', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, 
                              verbose=1, mode='auto')

In [ ]:
# Fitting the model
history = model.fit(x_train,
                    y_train,
                    epochs=n_epoch,
                    callbacks=[reduce_lr,early_stop],
                    validation_data=(x_val,y_val)
                   )
model.save('/content/drive/MyDrive/project models/xceptionskincancer.h5')

Epoch 1/300
254/254 [==============================] - 60s 90ms/step - loss: 1.2099 - accuracy: 0.6429 - val_loss: 1.0648 - val_accuracy: 0.7195
Epoch 2/300
254/254 [==============================] - 20s 80ms/step - loss: 0.7007 - accuracy: 0.7602 - val_loss: 1.7239 - val_accuracy: 0.6020
Epoch 3/300
254/254 [==============================] - 21s 82ms/step - loss: 0.5662 - accuracy: 0.8077 - val_loss: 0.8257 - val_accuracy: 0.7639
Epoch 4/300
254/254 [==============================] - 21s 84ms/step - loss: 0.4795 - accuracy: 0.8426 - val_loss: 0.7664 - val_accuracy: 0.7483
Epoch 5/300
254/254 [==============================] - 22s 86ms/step - loss: 0.3992 - accuracy: 0.8689 - val_loss: 0.7948 - val_accuracy: 0.7716
Epoch 6/300
254/254 [==============================] - 22s 86ms/step - loss: 0.3235 - accuracy: 0.8926 - val_loss: 0.7373 - val_accuracy: 0.7761
Epoch 7/300
254/254 [==============================] - 22s 85ms/step - loss: 0.2584 - accuracy: 0.9160 - val_loss: 0.7734 - val_ac

In [ ]:
import keras


model=keras.models.load_model('/content/drive/MyDrive/project models/xceptionskincancer.h5')



In [ ]:
y_test = to_categorical(y_test, num_classes = 7)
score, acc = model.evaluate(x_test,y_test,verbose=1,batch_size=64)

print('Test score:', score)
print('Test accuracy:', acc)

16/16 [==============================] - 24s 1s/step - loss: 0.6201 - accuracy: 0.8214
Test score: 0.6200746893882751
Test accuracy: 0.8213573098182678
